## Standardization for Darcy2d (141 x 141)

### Import Modules

In [1]:
import torch
import numpy as np
import time
from script.auxiliary import *
from script.ReadMatFiles import MatReader
from script.Normalization import get_pixelwise_standardization

### Measure Time

In [2]:
before_time = time.time()

### Load the Dataset

In [3]:
s = int(input('Resolution (241 or 421) : ')) #resolution
r = int(input('Downsampling Step : ')) #downsampling
assert (s-1) % r == 0
s_r = (s-1)//r + 1
print('Resolution : {} per each axis'.format(s_r))

Resolution (241 or 421) : 421
Downsampling Step : 3
Resolution : 141 per each axis


In [4]:
ntrain = 1000
ntest = 200

In [5]:
#Train Dataset : normalize both x & y
#Test Dataset : normalize only x (not y)
reader = MatReader('./dataset/original_dataset/piececonst_r' + str(s) + '_N1024_smooth1.mat')
u_train = reader.read_field('sol')[:ntrain,::r,::r][:,:s,:s] #(ntrain,s,s)
a_train = reader.read_field('coeff')[:ntrain,::r,::r][:,:s,:s] #(ntrain,s,s)

reader.load_file('./dataset/original_dataset/piececonst_r' + str(s) + '_N1024_smooth2.mat')
u_test = reader.read_field('sol')[:ntest,::r,::r][:,:s,:s] #(ntest,s,s)
a_test = reader.read_field('coeff')[:ntest,::r,::r][:,:s,:s] #(ntest,s,s)

### Standardization

In [6]:
u_train_standardized, u_train_mean, u_train_std = get_pixelwise_standardization(input_tensor=u_train, given_mean=None, given_std=None)
a_train_standardized, a_train_mean, a_train_std = get_pixelwise_standardization(input_tensor=a_train, given_mean=None, given_std=None)

In [7]:
u_test_standardized, _, _ = get_pixelwise_standardization(input_tensor=u_test, given_mean=u_train_mean, given_std=u_train_std)
a_test_standardized, _, _ = get_pixelwise_standardization(input_tensor=a_test, given_mean=a_train_mean, given_std=a_train_std)

### Save

In [8]:
base_path = './dataset/preprocessed_dataset/'

In [9]:
#u
torch.save(u_train_standardized, base_path+'u_train_standardized_{}.pt'.format(s_r))
torch.save(u_train_mean, base_path+'u_train_mean_{}.pt'.format(s_r))
torch.save(u_train_std, base_path+'u_train_std_{}.pt'.format(s_r))
torch.save(u_test_standardized, base_path+'u_test_standardized_{}.pt'.format(s_r))

In [10]:
#a
torch.save(a_train_standardized, base_path+'a_train_standardized_{}.pt'.format(s_r))
torch.save(a_train_mean, base_path+'a_train_mean_{}.pt'.format(s_r))
torch.save(a_train_std, base_path+'a_train_std_{}.pt'.format(s_r))
torch.save(a_test_standardized, base_path+'a_test_standardized_{}.pt'.format(s_r))

### Measure Time

In [11]:
after_time = time.time()
how_long = int(after_time - before_time)
print('{}hr {}min {}sec'.format(how_long//3600, (how_long%3600)//60, (how_long%3600)%60))

0hr 0min 30sec
